**Import required Modules, Packages, IPython**

Set-up Anaconda Virtual Environment for this entire project for the best results. GDAL Library works easily with Anaconda so it's better to use Anaconda Virtual Environment. Make sure to install the correct version of Python Anaconda which is compatible with Tensorflow, CuDA, cuDNN.   

In [10]:
# numeric packages
import numpy as np
import pandas as pd

# filesystem and OS
import sys, os, time
import glob

# plotting
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline

import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

# compression
import gzip
import pickle
import copy

# widgets and interaction
from IPython.display import display, clear_output

import warnings
warnings.filterwarnings('ignore')

Keep urbanatlas.py file in the same directory

In [2]:
# custom module for analyzing Urban Atlas data

import sys
import urbanatlas as ua

**Download Shapefile of the city**
1. Download the city's Urban Atlas data from [Urban Atlas Website](https://www.eea.europa.eu/en/datahub/datahubitem-view/e006507d-15c8-49e6-959c-53b61facd873)
2. Extract the ZIP folder of the city's data. Locate the .gpkg file after extraction in the Results folder. 
3. Use QGIS to export the .gpkg file into .shp shapefile. It is crucial to save the name of the .shp file in this format country_cityName. Eg: City: Berlin, Country: Germany, filename: de_berlin.shp

In [3]:
# path to shapefiles
shapefiles_path = r'C:\Users\lyq09mow\Copernicus Data\Berlin'

import re
import os
import glob

def fn_process_path(s):
    b = os.path.basename(s).split(".")[0]
    country = b.split("_")[0]
    city = " ".join(b.split("_")[1:])
    country = re.findall("[a-zA-Z]+", country)[0]
    return (city, country)

shapefiles = glob.glob(r"%s/**/*.shp" % shapefiles_path, recursive=True)
# shapefiles = glob.glob(os.path.join(shapefiles_path, "*.shp"))
shapefiles = {"%s, %s" % fn_process_path(f):f for f in shapefiles}

print(shapefiles_path)
print(shapefiles)

# path to save data
outPath = r"C:\Users\lyq09mow\Data\Berlin"

if not os.path.exists(outPath):
    os.makedirs(outPath)
    
# classess used in the Urban Atlas dataset

classes = '''Agricultural + Semi-natural areas + Wetlands
Airports
Construction sites
Continuous Urban Fabric (S.L. > 80%)
Discontinuous Dense Urban Fabric (S.L. : 50% -  80%)
Discontinuous Low Density Urban Fabric (S.L. : 10% - 30%)
Discontinuous Medium Density Urban Fabric (S.L. : 30% - 50%)
Discontinuous Very Low Density Urban Fabric (S.L. < 10%)
Fast transit roads and associated land
Forests
Green urban areas
Isolated Structures
Land without current use
Mineral extraction and dump sites
Other roads and associated land
Port areas
Railways and associated land
Sports and leisure facilities
Water bodies'''.split("\n")

class2label = {c:i for i,c in enumerate(classes)}
label2class = {i:c for i,c in enumerate(classes)}

C:\Users\lyq09mow\Copernicus Data\Berlin
{'berlin, de': 'C:\\Users\\lyq09mow\\Copernicus Data\\Berlin\\de_berlin.shp'}


In [4]:
myname = "berlin, de"

# read in shapefile
shapefile = shapefiles[myname]

mycity = ua.UAShapeFile(shapefile, name=myname)

L = mycity.compute_spatial_extent()
print("Spatial extent: %2.2f km." % L)

classified_pct = mycity.compute_classified_area()
print(classified_pct)

Columns in the shapefile: Index(['ID', 'OBJECTID', 'COUNTRY', 'FUA_NAME', 'FUA_CODE', 'CODE_2018',
       'CLASS_2018', 'PROD_DATE', 'IDENTIFIER', 'COMMENT', 'SHAPE_LENG',
       'SHAPE_AREA', 'geometry', 'SHAPE_LEN'],
      dtype='object')
88079 polygons | 25 land use classes
Spatial extent: 41.61 km.
CLASS_2018
Airports                                                                            0.009765
Arable land (annual crops)                                                          0.224418
Complex and mixed cultivation patterns                                              0.000160
Construction sites                                                                  0.002608
Continuous urban fabric (S.L. : > 80%)                                              0.105416
Discontinuous dense urban fabric (S.L. : 50% -  80%)                                0.322675
Discontinuous low density urban fabric (S.L. : 10% - 30%)                           0.037631
Discontinuous medium density urban

In [7]:
lonmin, latmin, lonmax, latmax = mycity._bounds
city_center = ((latmin+latmax)/2.0, (lonmin+lonmax)/2.0)
mycity_crop = mycity.crop_centered_window(city_center, (25,25))

print(mycity_crop._gdf.shape, mycity._gdf.shape)

(68999, 14) (88079, 14)


In [8]:
raster, locations, cur_classes = mycity_crop.extract_class_raster(grid_size=(25,25))
print(raster.shape)

# (58168, 14) (362415, 14)

(25, 25, 24)


In [9]:
locations_train = mycity.generate_sampling_locations()
locations_train.head()

Initial gdf_sel columns: Index(['ID', 'OBJECTID', 'COUNTRY', 'FUA_NAME', 'FUA_CODE', 'CODE_2018',
       'CLASS_2018', 'PROD_DATE', 'IDENTIFIER', 'COMMENT', 'SHAPE_LENG',
       'SHAPE_AREA', 'geometry', 'SHAPE_LEN'],
      dtype='object')
Initial gdf_sel index names: [None]
--------------------------------------------------
After grouping, select_polygons columns: Index(['ID', 'OBJECTID', 'COUNTRY', 'FUA_NAME', 'FUA_CODE', 'CODE_2018',
       'CLASS_2018', 'PROD_DATE', 'IDENTIFIER', 'COMMENT', 'SHAPE_LENG',
       'SHAPE_AREA', 'geometry', 'SHAPE_LEN', 'samples'],
      dtype='object')
After grouping, select_polygons index names: ['CLASS_2018', None]
--------------------------------------------------
Before renaminng, select_polygons columns: Index(['ID', 'OBJECTID', 'COUNTRY', 'FUA_NAME', 'FUA_CODE', 'CODE_2018',
       'CLASS_2018', 'PROD_DATE', 'IDENTIFIER', 'COMMENT', 'SHAPE_LENG',
       'SHAPE_AREA', 'geometry', 'SHAPE_LEN', 'samples'],
      dtype='object')
After renaming, sele

lon        lat
ITEM_CLASS                                        
Airports                   0  13.287729  52.558385
                           1  13.287729  52.558385
                           2  13.287729  52.558385
                           3  13.287729  52.558385
Arable land (annual crops) 0  13.473998  52.408855

Run pipeline for all cities in the Urban Atlas dataset

In [11]:
# %%px --local

# for constructing ground truth raster grids

grid_cell = 100
grid_size = (grid_cell, grid_cell)
window_km = 25

# for generating sampling locations

img_area = (224 * 1.19/ 1000)**2 # in km^2, at zoom level 17
thresh_frac = 0.25 # at least <thresh_frac> % of the image should be covered by a polygon of a given class
thresh_area = img_area * thresh_frac  
# print "Threshold area: %2.2f km^2"%thresh_area

n_classes = len(classes)

N_SAMPLES_PER_CITY  = 25000
N_SAMPLES_PER_CLASS = N_SAMPLES_PER_CITY / n_classes
MAX_SAMPLES_PER_POLY= 50

In [15]:
def fn_generate_stats(shapefile):
    cityname = "%s, %s" % fn_process_path(shapefile)
        
    print("Processing %s"%cityname)
    
    savedir = "%s/%s/"%(outPath, cityname)
    if not os.path.exists(savedir):
        os.makedirs(savedir)

            #if len([x for x in os.listdir(savedir) if 'ground_truth_class_raster' in x])>=1:
            #return "Already processed!"
   
    mycity = ua.UAShapeFile(shapefile, name=cityname)
    
    if mycity._gdf is None:
        return "Error reading shapefile %s"%shapefile
     
    # approximate city center by the center of the bounding box of the shapefile
    lonmin, latmin, lonmax, latmax = mycity._bounds
    city_center = ((latmin+latmax)/2.0, (lonmin+lonmax)/2.0)

    # there's some weird issue with the shapefile for Graz
    # lat and lon are inverted?
    if cityname in ["graz, at"]: #not bounds_gdf.contains(Point(city_center[::-1])):
        gdf['geometry'] = gdf['geometry'].apply(\
                lambda p: Polygon((lon,lat) \
                    for (lon,lat) in zip(p.exterior.coords.xy[1], p.exterior.coords.xy[0])))
    
    # compute spatial extent of city and fraction of land classified
    L = mycity.compute_spatial_extent()
    frac_classified = mycity.compute_classified_area()
    frac_classified['pct land classified'] = frac_classified.sum()
    frac_classified['spatial extent'] = L
    frac_classified.to_csv("%s/basic_stats.csv"%savedir)
        
    window = (window_km, window_km)
    mycity_crop = mycity.crop_centered_window(city_center, window)

    # compute ground traster for given window size
    raster, locations_grid, cur_classes = mycity_crop.extract_class_raster(grid_size=grid_size)
    myraster = np.zeros(grid_size + (len(classes),))
    # idx = [class2label[c] for k,c in enumerate(cur_classes)]
    # myraster[:,:,idx] = raster

    # Filter out classes not in class2label
    valid_classes = [c for c in cur_classes if c in class2label]
    idx = [class2label[c] for c in valid_classes]

    print("PRINTING IDX:\n")
    print(idx)

    for i, c in enumerate(valid_classes):
        myraster[:, :, idx[i]] = raster[:, :, i]

    # save data
    np.savez_compressed("%s/ground_truth_class_raster_%dkm.npz"%(savedir,window_km), myraster, classes)

    # Reverse class2label dictionary for mapping back to class names
    label2class = {v: k for k, v in class2label.items()}

    # Replace class numbers with class names in locations_grid
    locations_grid['class'] = locations_grid['class'].map(label2class)
    
    locations_grid.to_csv("%s/sample_locations_raster_%dkm.csv"%(savedir,window_km), index=False)
 
    # extract sampling locations for training
    locations_train = mycity.generate_sampling_locations(thresh_area=thresh_area, \
                                                         n_samples_per_class=N_SAMPLES_PER_CLASS,\
                                                         max_samples=MAX_SAMPLES_PER_POLY)
    locations_train.to_csv("%s/additional_sample_locations.csv"%(savedir))

In [14]:
shapefile = shapefiles["berlin, de"]
cityname = "%s, %s" % fn_process_path(shapefile)
print(cityname)
savedir = r'C:\Users\lyq09mow\Copernicus Data\Madrid\new'
print(len([x for x in os.listdir(savedir) if 'ground_truth_class_raster' in x])==3)
[x for x in os.listdir(savedir) if 'ground_truth_class_raster' in x]

print("-"*50)
print(shapefile)

raster, locations_grid, cur_classes = mycity_crop.extract_class_raster(grid_size=grid_size)

print("-"*50)
print("Cur Classes")
print(cur_classes)

print("-"*50)
print("Raster")
print(raster)

print("-"*50)
print("Location grid: ")
print(locations_grid)


berlin, de
False
--------------------------------------------------
C:\Users\lyq09mow\Copernicus Data\Berlin\de_berlin.shp
--------------------------------------------------
Cur Classes
['Discontinuous medium density urban fabric (S.L. : 30% - 50%)'
 'Discontinuous dense urban fabric (S.L. : 50% -  80%)'
 'Other roads and associated land' 'Pastures'
 'Industrial, commercial, public, military and private units'
 'Herbaceous vegetation associations (natural grassland, moors...)'
 'Railways and associated land' 'Green urban areas'
 'Land without current use' 'Fast transit roads and associated land'
 'Continuous urban fabric (S.L. : > 80%)'
 'Discontinuous low density urban fabric (S.L. : 10% - 30%)'
 'Construction sites' 'Arable land (annual crops)'
 'Sports and leisure facilities' 'Forests' 'Isolated structures' 'Water'
 'Wetlands' 'Mineral extraction and dump sites'
 'Discontinuous very low density urban fabric (S.L. : < 10%)' 'Port areas'
 'Complex and mixed cultivation patterns' 'Airp

In [16]:
fn_generate_stats(shapefile)

Processing berlin, de
Columns in the shapefile: Index(['ID', 'OBJECTID', 'COUNTRY', 'FUA_NAME', 'FUA_CODE', 'CODE_2018',
       'CLASS_2018', 'PROD_DATE', 'IDENTIFIER', 'COMMENT', 'SHAPE_LENG',
       'SHAPE_AREA', 'geometry', 'SHAPE_LEN'],
      dtype='object')
88079 polygons | 25 land use classes
PRINTING IDX:

[14, 16, 10, 12, 8, 2, 17, 9, 13, 15, 1]
Initial gdf_sel columns: Index(['ID', 'OBJECTID', 'COUNTRY', 'FUA_NAME', 'FUA_CODE', 'CODE_2018',
       'CLASS_2018', 'PROD_DATE', 'IDENTIFIER', 'COMMENT', 'SHAPE_LENG',
       'SHAPE_AREA', 'geometry', 'SHAPE_LEN'],
      dtype='object')
Initial gdf_sel index names: [None]
--------------------------------------------------
After grouping, select_polygons columns: Index(['ID', 'OBJECTID', 'COUNTRY', 'FUA_NAME', 'FUA_CODE', 'CODE_2018',
       'CLASS_2018', 'PROD_DATE', 'IDENTIFIER', 'COMMENT', 'SHAPE_LENG',
       'SHAPE_AREA', 'geometry', 'SHAPE_LEN', 'samples'],
      dtype='object')
After grouping, select_polygons index names: ['CLA